<a href="https://colab.research.google.com/github/musicjae/NLP/blob/master/from_scratch_2/1_intro_NLP_and_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 자연어 처리  
  
- 우리 인간이 사용하는 언어를 지칭하는 용어를 '자연(언)어'라고 하자. 이것은 기계가 사용하는 언어를 지칭하는 용어 '인공(언)어'와 구분된다. 후자는 기계적이고 고정된 언어라고 간주된다면, 전자의 것은 살아있는, 부드러운 것이라고 간주된다. 가령 명사 '고구마'는 우리가 흔히 알고 있는 그 먹는 고구마를 지칭하기도 하지만, 최근에는 답답하고 거북한 어떤 상황을 지칭하여 '고구마스럽다'라고 사용되기도 한다. 이 사례는 이전에 사용된 적 없던 표현이 새로운 상황에서 사용된다는 점에서 언어는 살아있고 부드럽게 사용되고 있음을 함축한다.  
  
- 우리가 NLP에서 할 일은 우리가 사용하는 자연어를 컴퓨터에게 인공어를 이용하여 자연어 내의 단어의 의미를 이해시키는 것이다. 어떻게 컴퓨터가 우리가 사용하는 자연어를 이해하는가? 이 물음에 대한 답은 3 가지 방식으로 제시될 수 있다:  
  - Thesaurus(유의어 사전)  
  - 통계 기반 기법  
  - 추론 기반 기법

## 1.2 Thesaurus  
 
- 시소러스는 사람이 수작업으로 만든 유의어 사전을 이용하는 것이다. 이 유의어 사전은 단어의 존재론적 상하 관계를 이용하여 유의어들을 관리한다. 가령, 최상위 단어는 object를 사용할 때, 그것의 부분집합 중 하나로 원동기를 뜻하는 motor vehicle를 포함시킬 수 있고, 원동기 아래에 포함된 부분 집합 단어들은 '자동차', '트럭' 등이 있을 수 있다. 이 **단어 네트워크**를 이용하여 컴퓨터에게 단어 간의 관계를 학습시키는 것이 시소러스를 이용한 방법이다.

### 1.2.1 WordNet  
  
- 시소러스를 이용하는 대표적인 방법은 WordNet을 이용하는 것이다.  
- 참고: https://www.nltk.org/howto/wordnet.html

In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [9]:
from nltk.corpus import wordnet
wordnet.synsets('car')

[Synset('car.n.01'),
 Synset('car.n.02'),
 Synset('car.n.03'),
 Synset('car.n.04'),
 Synset('cable_car.n.01')]

'car'가 5 가지의 의미를 갖고 있음을 확인. n은 명사임을 뜻하고, 번호는 인덱스를 뜻함.

In [13]:
mind=wordnet.synset('mind.n.01')
mind.definition()


"that which is responsible for one's thoughts and feelings; the seat of the faculty of reason"

위의 것은 'mind'의 첫 번째 단어의 정의를 시소러스 사전에서 출력한 것이다. 다음으로, 'mind'의 동의어 사전을 검색해보자.

In [14]:
mind.lemma_names()

['mind', 'head', 'brain', 'psyche', 'nous']

'mind'의 상하 관계를 살펴보자:

In [15]:
mind.hypernym_paths()

[[Synset('entity.n.01'),
  Synset('abstraction.n.06'),
  Synset('psychological_feature.n.01'),
  Synset('cognition.n.01'),
  Synset('mind.n.01')]]

단어 'mind', 'head', 'stone'의 단어 간 유사도를 측정해보자.

In [17]:
mind = wordnet.synset('mind.n.01')
head = wordnet.synset('head.n.01')
stone = wordnet.synset('stone.n.01')

print('mind--head: ', mind.path_similarity(head))

print('mind--stone: ', mind.path_similarity(stone))



mind--head:  0.09090909090909091
mind--stone:  0.1


### 1.2.2 시소러스의 문제점

- 시대 변화에 대응하기 어렵다.   
  - 자연어의 단어는 살아있고 유연하다. 이것은 시대에 따라 다른 의미로 사용될 수 있다. 시소러스는 그러한 변화에 민감하게 반응하여 그것에 맞추어 사람이 시소러스를 수정하게끔 한다.  
- 사람을 쓰는 비용은 비싸다.  
  - 첫 번째 곤경 같은 경우에 사람을 사용하여 단어의 의미를 수정해야 하는데 이때 요구되는 인력 비용은 비싸다.  
- 단어 간의 미묘한 차이를 표현하지 못한다.  
  - 가령, 단어 'mind'는 물리적인 것만으로 구성된 인간의 몸과 대비되는 의미로 '마음'이라고 표현되기도 하지만, 일상적인 의미에서 '호의를 베푼다'를 표현하기 위해 '마음(을 쓰다)'라고 사용되기도 한다. 시소러스는 이런 단어의 미묘한 차이를 표현하기 어렵고, 이것이 가능하기 위해서는 많은 인력 사용의 대가를 치뤄야 한다.

# 2 통계 기반 기법  
  
- 말뭉치corpus: nlp 연구를 목적으로 두고 수집된 텍스트 데이터. 이것은 사람이 지식, 정보를 담은 자연어로 구성되어 있다.   
  
통계 기반 기법에서는 이 말뭉치corpus를 자동으로 추출하여 다룬다.

## 2.1 전처리 Preprocessing  
  
- txt 파일 같은 것으로 입력된 말뭉치는 학습하는 데 사용되기 위해 전처리 작업을 거쳐야 한다. 이것은 이 말뭉치를 (1) 단어로 분할, (2) 그것들에 ID를 부착하는 작업이라고 요약되어질 수 있다.

In [24]:
corpus = 'Philosophy is based on natural language. But Mathematics can be not based on it.'

# (1) 소문자화
corpus=corpus.lower()
# (2) 마침표 앞에 공백 넣기
corpus=corpus.replace('.',' .')
# (3) 공백을 기준으로 말뭉치를 단어로 분할하기
words = corpus.split(' ')
print(words)

['philosophy', 'is', 'based', 'on', 'natural', 'language', '.', 'but', 'mathematics', 'can', 'be', 'not', 'based', 'on', 'it', '.']


  
(1) word2id: 단어에 id 부여  
  
(2) id2word: id의 리스트를 이용하여 단어에 접근할 수 있도록 해준다.

In [25]:
word2id={} # dict 형태로 초기화
id2word={}

for word in words:
    if word not in word2id:
        new_id = len(word2id) # 우변은 0 - 12까지 늘어난다.
        word2id[word]=new_id # 처음 0 idx 경우, word2id[word]에는 입력되는 첫 단어 'philosophy'가 0 idx와 상응하여 저장. 12까지 차례대로 단어들 들어오며 이 과정 반복
        id2word[new_id]=word # idx 0부터 상응하는 각 단어를 dict 형태로 출력
        
print(id2word)
print(word2id)


{0: 'philosophy', 1: 'is', 2: 'based', 3: 'on', 4: 'natural', 5: 'language', 6: '.', 7: 'but', 8: 'mathematics', 9: 'can', 10: 'be', 11: 'not', 12: 'it'}
{'philosophy': 0, 'is': 1, 'based': 2, 'on': 3, 'natural': 4, 'language': 5, '.': 6, 'but': 7, 'mathematics': 8, 'can': 9, 'be': 10, 'not': 11, 'it': 12}


단어 리스트를 단어 id 리스트로 변경해보자!

In [26]:
import numpy as np
corpus = [word2id[w] for w in words] # comprehension -> store them as list type
corpus = np.array(corpus) # list->np
print(corpus,type(corpus))

[ 0  1  2  3  4  5  6  7  8  9 10 11  2  3 12  6] <class 'numpy.ndarray'>


위 작업을 함수화 해보자.

In [33]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    text = text.split(' ')

    word2id={}
    id2word={}

    for word in words:
        if word not in word2id:
            new_id=len(word2id)
            word2id[word]=new_id
            id2word[new_id]=word

    text = np.array([word2id[w] for w in words])

    return text, word2id, id2word


참고: 사이토 코기, '밑바닥부터 시작하는 딥러닝2', 한빛미디어